##  A neural Network replacing the simple xgboost regression algorithm

#### Importing all libraries and setting the device to gpu

In [16]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error
from torch.utils.data import Dataset, DataLoader

import seaborn as sns
import random

# torch.cuda.set_device("cuda:0")
# print(torch.cuda.get_device_name())

In [17]:
import random
def set_seeds(seed=1234):
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # multi-GPU
    
set_seeds()

#### Pre processing the data and adding data indicators to the data

In [18]:
df = pd.read_csv("../data/reliance/reliance.csv")
# Need to reverse the db
# df = df.loc[::-1]
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True,format="mixed")
df = df[(df['Date'].dt.year >= 2020)].copy()
df.index = range(len(df))
temp = df
# df['Close'] = (df['High'] + df['Low'] )/ 2
df.rename(columns={"Open Price":"Open","High Price":"High","Low Price":"Low","Close Price":"Close","Total Traded Quantity":"Volume","No.of Shares":"Volume","Price":"Close"},inplace=True)
# cols = ["Symbol","Ser verbose=Falseies","Prev Close","Last Price","Average Price","Turnover","No. of Trades", "Deliverable Qty",'% Dly Qt to Traded Qty']
# cols = ["WAP","No. of Trades"	,"Total Turnover (Rs.)"	,"Deliverable Quantity"	,"% Deli. Qty to Traded Qty"	,"Spread High-Low"	,"Spread Close-Open"]
cols = "Adj Close"
# # cols = "Change(%)"
df.drop(columns=cols,inplace=True)   
df.dropna(inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume
0,2020-01-17,1538.912598,1570.067383,1538.615479,1566.154419,13597387
1,2020-01-20,1593.891479,1593.891479,1512.067139,1517.961304,15019904
2,2020-01-21,1514.246460,1531.334473,1507.708374,1519.496704,8732832
3,2020-01-22,1529.501831,1532.226074,1516.723022,1518.951904,4763978
4,2020-01-23,1522.072266,1527.471069,1506.420654,1512.512939,5190829


In [19]:
df['EMA_9'] = df['Close'].ewm(9).mean().shift()
df['SMA_5'] = df['Close'].rolling(5).mean().shift()
df['SMA_10'] = df['Close'].rolling(10).mean().shift()
df['SMA_15'] = df['Close'].rolling(15).mean().shift()
df['SMA_30'] = df['Close'].rolling(30).mean().shift()

def relative_strength_idx(df, n=14):
    close = df['Close']
    delta = close.diff()
    delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0    
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

df['RSI'] = relative_strength_idx(df).fillna(0)

EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
df['MACD'] = pd.Series(EMA_12 - EMA_26)
df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean())

df['Close'] = df['Close'].shift(-1)

df = df.iloc[33:] # Because of moving average
df = df[:-1]      # Because of shifting close price

df.index = range(len(df))

df.head()

,Date,Open,High,Low,Close,Volume,EMA_9,SMA_5,SMA_10,SMA_15,SMA_30,RSI,MACD,MACD_signal
0,2020-03-05,1339.502808,1345.198853,1293.736694,1259.065308,14080540,1394.011439,1330.111865,1382.564661,1408.938005,1428.786536,24.217013,-33.317728,-23.973070
1,2020-03-06,1258.074707,1276.054321,1229.446045,1103.688110,15568253,1384.221844,1315.232886,1363.480554,1398.427629,1421.449414,18.407224,-38.450364,-27.216825
2,2020-03-09,1228.405884,1229.347046,1084.718018,1142.718140,40698294,1371.384861,1303.811157,1342.187390,1385.011467,1413.001160,12.869663,-53.706911,-33.012704
3,2020-03-11,1097.595947,1167.384399,1097.595947,1053.018433,27495503,1343.998209,1263.790503,1309.418005,1360.348576,1399.548674,19.755222,-62.077352,-39.254572
4,2020-03-12,1074.811890,1090.661621,1038.852661,1094.921265,25830369,1323.453649,1226.286011,1283.379822,1338.905168,1387.892493,11.535767,-74.716404,-46.759527


In [20]:
drop_cols = ['Date', 'Volume', 'Open', 'Low', 'High']
df.drop(columns=drop_cols,inplace=True)
df.head()

,Close,EMA_9,SMA_5,SMA_10,SMA_15,SMA_30,RSI,MACD,MACD_signal
0,1259.065308,1394.011439,1330.111865,1382.564661,1408.938005,1428.786536,24.217013,-33.317728,-23.973070
1,1103.688110,1384.221844,1315.232886,1363.480554,1398.427629,1421.449414,18.407224,-38.450364,-27.216825
2,1142.718140,1371.384861,1303.811157,1342.187390,1385.011467,1413.001160,12.869663,-53.706911,-33.012704
3,1053.018433,1343.998209,1263.790503,1309.418005,1360.348576,1399.548674,19.755222,-62.077352,-39.254572
4,1094.921265,1323.453649,1226.286011,1283.379822,1338.905168,1387.892493,11.535767,-74.716404,-46.759527


#### Separating the data into train and test splits. Then splitting into validation split

In [21]:
X = df.iloc[:, 1:]
y = df.iloc[:, 1]

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.2
)

#### Normalising the data

In [22]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

#### Initialising the dataset

In [23]:
class RegressionDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)   

In [24]:
train_dataset = RegressionDataset(
    torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()
)
val_dataset = RegressionDataset(
    torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float()
)
test_dataset = RegressionDataset(
    torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()
)

#### Setting hyper parameters

In [25]:
EPOCHS = 850
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_FEATURES = len(X.columns)

In [26]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE,)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE)

In [27]:
class MultipleRegression(nn.Module):
    def __init__(self, num_features):
        super(MultipleRegression, self).__init__()

        self.layer_1 = nn.Linear(num_features, 16)
        self.layer_2 = nn.Linear(16, 32)
        self.layer_3 = nn.Linear(32, 16)
        self.layer_out = nn.Linear(16, 1)

        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.layer_out(x)
        return x

    def predict(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.layer_out(x)
        return x
# torch.cuda.set_device("cuda:0")
# print(torch.cuda.get_device_name())

In [28]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
model = MultipleRegression(NUM_FEATURES)
model.to(device)
print(model)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

MultipleRegression(
  (layer_1): Linear(in_features=8, out_features=16, bias=True)
  (layer_2): Linear(in_features=16, out_features=32, bias=True)
  (layer_3): Linear(in_features=32, out_features=16, bias=True)
  (layer_out): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)


In [29]:
loss_stats = {
    'train': [],
    "val": []
}

In [30]:
for e in tqdm(range(1, EPOCHS + 1)):
    # TRAINING
    train_epoch_loss = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(
            device
        )
        optimizer.zero_grad()

        y_train_pred = model(X_train_batch)

        train_loss = criterion(y_train_pred, y_train_batch.unsqueeze(1))

        train_loss.backward()
        optimizer.step()

        train_epoch_loss += train_loss.item()

    # VALIDATION
    with torch.no_grad():
        val_epoch_loss = 0

        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

            y_val_pred = model(X_val_batch)

            val_loss = criterion(y_val_pred, y_val_batch.unsqueeze(1))

            val_epoch_loss += val_loss.item()
        # print(torch.cuda.get_device_name())

        loss_stats["train"].append(train_epoch_loss / len(train_loader))
        loss_stats["val"].append(val_epoch_loss / len(val_loader))

    if e % 50 == 0:
        print(
            f"Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f}"
        )
# torch.cuda.set_device("cuda:0")
# print(torch.cuda.get_device_name())

  0%|          | 0/850 [00:00<?, ?it/s]

  8%|▊         | 70/850 [00:07<00:30, 25.74it/s] 

Epoch 050: | Train Loss: 32954.25391 | Val Loss: 31609.59896


 14%|█▍        | 122/850 [00:08<00:10, 67.31it/s]

Epoch 100: | Train Loss: 27612.16254 | Val Loss: 25189.08854


 20%|██        | 174/850 [00:08<00:06, 105.12it/s]

Epoch 150: | Train Loss: 22153.76497 | Val Loss: 19683.82910


 25%|██▌       | 214/850 [00:08<00:05, 120.22it/s]

Epoch 200: | Train Loss: 17198.92708 | Val Loss: 14808.24154


 31%|███▏      | 267/850 [00:09<00:04, 125.44it/s]

Epoch 250: | Train Loss: 12848.80257 | Val Loss: 10984.61263


 38%|███▊      | 321/850 [00:09<00:04, 127.86it/s]

Epoch 300: | Train Loss: 9616.11887 | Val Loss: 7952.71883


 44%|████▍     | 374/850 [00:09<00:03, 128.90it/s]

Epoch 350: | Train Loss: 6663.11182 | Val Loss: 5521.53349


 49%|████▊     | 414/850 [00:10<00:03, 125.16it/s]

Epoch 400: | Train Loss: 4233.66075 | Val Loss: 3478.30652


 55%|█████▍    | 467/850 [00:10<00:03, 124.23it/s]

Epoch 450: | Train Loss: 2247.76010 | Val Loss: 1832.81366


 61%|██████    | 519/850 [00:11<00:02, 124.00it/s]

Epoch 500: | Train Loss: 970.21402 | Val Loss: 746.92798


 67%|██████▋   | 572/850 [00:11<00:02, 127.69it/s]

Epoch 550: | Train Loss: 280.86428 | Val Loss: 217.49014


 74%|███████▎  | 625/850 [00:12<00:01, 123.86it/s]

Epoch 600: | Train Loss: 55.69100 | Val Loss: 43.42832


 78%|███████▊  | 664/850 [00:12<00:01, 121.77it/s]

Epoch 650: | Train Loss: 11.63810 | Val Loss: 8.93807


 84%|████████▍ | 716/850 [00:12<00:01, 124.27it/s]

Epoch 700: | Train Loss: 6.41264 | Val Loss: 5.14860


 90%|█████████ | 768/850 [00:13<00:00, 119.88it/s]

Epoch 750: | Train Loss: 6.01941 | Val Loss: 5.15719


 97%|█████████▋| 821/850 [00:13<00:00, 126.91it/s]

Epoch 800: | Train Loss: 5.91416 | Val Loss: 5.03936


100%|██████████| 850/850 [00:13<00:00, 61.23it/s] 

Epoch 850: | Train Loss: 5.86809 | Val Loss: 4.85864


In [31]:
print(len(loss_stats['train']))
print(len(loss_stats['val']))

850
850


Text(0.5, 1.0, 'Train-Val Loss/Epoch')

In [32]:
train_val_loss_df = (
    pd.DataFrame.from_dict(loss_stats)
    .reset_index()
    .melt(id_vars=["index"])
    .rename(columns={"index": "epochs"})
)
plt.figure(figsize=(15, 8))
sns.lineplot(data=train_val_loss_df, x="epochs", y="value", hue="variable").set_title(
    "Train-Val Loss/Epoch"
)

100%|██████████| 3/3 [00:00<00:00, 27962.03it/s]

[2517.5447, 2598.6392, 2531.5276, 2218.9954, 1942.6741, 1997.3632, 2437.3381, 2393.6838, 2429.386, 1793.1844, 2301.9817, 2027.99, 1095.8844, 1966.2649, 2451.6558, 2012.4957, 2096.6372, 2226.469, 2446.7021, 2036.9781, 1981.0187, 1483.4465, 2565.2754, 2065.2344, 1957.9697, 2670.9917, 1847.9707, 2569.3914, 2428.5042, 2475.4292, 2218.1982, 2176.1306, 2591.0564, 2008.0157, 2177.826, 2591.8625, 2033.5092, 2589.674, 2611.9546, 2091.9907, 2441.8003, 1958.4121, 2131.038, 2435.1548, 2388.5132, 1984.1823, 2521.2275, 2486.4504, 2512.6543, 1340.0905, 2191.2554, 2141.6729, 1947.2583, 2501.9343, 1963.5543, 2513.9453, 1980.6389, 2084.7346, 2511.4756, 2570.737, 2628.1177, 2420.9492, 2620.7356, 2614.852, 1193.5764, 2032.4366, 2146.4226, 2113.6567, 1658.3864, 2426.9932, 2359.6052, 1970.1582, 1604.4236, 2572.3425, 2558.1868, 2457.1255, 2651.6606, 2411.3682, 2568.0063, 2077.1433, 2699.433, 2421.263, 2089.7388, 2473.0356, 2408.169, 1152.8308, 2388.4878, 2671.5881, 2422.8296, 2382.526, 2214.1992, 2580.7808, 

[Text(0.5, 1.0, 'Deviation of Error')]

In [33]:
y_pred_list = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_pred_list.append(y_test_pred.cpu().numpy())
        y_pred_list = [a for a in y_pred_list]
        
y_pred = []
for i in tqdm(range(len(y_pred_list))):
    for j in range(len(y_pred_list[i])):
        for k in range(len(y_pred_list[i][j])):
            y_pred.append(y_pred_list[i][j][k])

print(y_pred)  
print(y_test.tolist())
y_pred_np = np.array(y_pred)
difference = np.subtract(y_pred_np, y_test)
difference = abs(difference)
mean_difference = np.mean(difference)
print(f"The average deviation in error is: {mean_difference}")
sns.lineplot(difference).set(title="Deviation of Error")

In [34]:
mse = mean_squared_error(y_test, y_pred)
r_square = r2_score(y_test, y_pred)
mean_abs_err = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Squared Error :",mse)
print("R^2 :",r_square)
print(f"Accuracy (using MSE): {(100 - mse)}%")
print(f"Mean absolute percentage error: {100 - mean_abs_err}%")

Mean Squared Error : 6.731599208270174
R^2 : 0.9999458797743397
Accuracy (using MSE): 93.26840079172983%
Mean absolute percentage error: 99.99906636461597%


#### Visualising the data

In [35]:
import plotly as py
import plotly.io as pio    
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [39]:
fig = make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(x=temp.index // 15, y=df.Close,
                         name='Truth',
                         marker_color='LightSkyBlue'), row=1, col=1)


fig.add_trace(go.Scatter(x=(len(temp) // 15 + (temp.index // 15)),
                         y=y_pred,
                         name='Prediction',
                         marker_color='MediumPurple'), row=2, col=1)

fig.add_trace(go.Scatter(x=(len(temp) // 15 + (temp.index // 15)),
                         y=y_test,
                         name='Truth',
                         marker_color='LightSkyBlue',
                         showlegend=False), row=3, col=1)



fig.show()